In [26]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

class CarAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.parked = False
        self.parked_steps = 0

    def move_to_parking(self):
        """ Move to an adjacent parking space if available. """
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        parking_spaces = []
        for pos in neighborhood:
            cellmates = self.model.grid.get_cell_list_contents(pos)
            for mate in cellmates:
                if isinstance(mate, ParkingSpaceAgent) and not mate.occupied:
                    parking_spaces.append(pos)
        return random.choice(parking_spaces) if parking_spaces else None

    def move(self):
        """ Move to a new, random position, prioritizing parking spaces. """
        new_position = self.move_to_parking()
        if new_position is not None:
            self.model.grid.move_agent(self, new_position)
        else:
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            new_position = random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)

    def find_parking(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        for mate in cellmates:
            if isinstance(mate, ParkingSpaceAgent) and not mate.occupied:
                mate.occupied = True
                self.parked = True
                self.parked_steps = random.randint(3, 5)
                break

    def step(self):
        if self.parked:
            if self.parked_steps > 0:
                # Decrement the parked steps and return early to stay parked
                self.parked_steps -= 1
                return
            else:
                # Free up the parking space
                cellmates = self.model.grid.get_cell_list_contents([self.pos])
                for mate in cellmates:
                    if isinstance(mate, ParkingSpaceAgent):
                        mate.occupied = False

                # Remove car from the grid and the schedule
                self.model.grid.remove_agent(self)
                self.model.schedule.remove(self)

        else:
            # If not parked, try to move and find parking
            self.move()
            self.find_parking()

class ParkingSpaceAgent(Agent):
    """ An agent representing a parking space. """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.occupied = False

    def step(self):
        pass  # Parking spaces do not have a step method

class ParkingLotModel(Model):
    """ A model representing a parking lot with multiple cars and parking spaces. """
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, False)
        self.schedule = RandomActivation(self)

        # Create parking spaces without overlap
        parking_space_positions = set()
        while len(parking_space_positions) < 5:
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            if (x, y) not in parking_space_positions:
                parking_space = ParkingSpaceAgent(len(parking_space_positions), self)
                self.grid.place_agent(parking_space, (x, y))
                self.schedule.add(parking_space)
                parking_space_positions.add((x, y))

        # Create cars
        for i in range(N):
            car = CarAgent(i + 100, self)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(car, (x, y))
            self.schedule.add(car)

        self.datacollector = DataCollector(
            agent_reporters={"Position": "pos", "Parked": "parked"}
        )

    def step(self):
        """ Advance the model by one step. """
        self.datacollector.collect(self)
        self.schedule.step()


In [27]:
import matplotlib.pyplot as plt
import numpy as np

def draw_model_state(model):
    # Define colors
    road_color = [255, 255, 255]  # White
    car_color = [0, 0, 255]       # Blue
    parking_color = [0, 255, 0]   # Green
    occupied_color = [255, 0, 0]  # Red

    # Initialize grid
    grid = np.zeros((model.grid.width, model.grid.height, 3), dtype=np.uint8) + road_color

    # Iterate over every cell in the grid
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            contents = model.grid.get_cell_list_contents((x, y))
            contains_car = any(isinstance(agent, CarAgent) for agent in contents)
            contains_space = any(isinstance(agent, ParkingSpaceAgent) for agent in contents)
            is_occupied = contains_car and contains_space

            if is_occupied:
                grid[y, x] = occupied_color
            elif contains_car:
                grid[y, x] = car_color
            elif contains_space:
                grid[y, x] = parking_color

    return grid

# Parameters for the simulation
num_cars = 10
width, height = 10, 10
num_steps = 20

# Create and run the model
model = ParkingLotModel(num_cars, width, height)

for i in range(num_steps):
    model.step()
    grid_image = draw_model_state(model)
    plt.imshow(grid_image, interpolation='nearest')
    plt.axis('off')
    plt.title(f'Step {i}')
    plt.savefig(f'parking_lot_step_{i}.png', bbox_inches='tight')
    plt.close()


Exception: Table does not exist.

In [ ]:
for i in range(num_steps):
    model.step()

# Generate bar charts
parking_time_data = model.datacollector.get_table_dataframe("ParkingTime")
plt.figure(figsize=(10, 6))

for agent_id in range(100, 100 + num_cars):
    agent_data = parking_time_data[parking_time_data['AgentID'] == agent_id]
    if not agent_data.empty:
        plt.bar(str(agent_id), agent_data['Steps'].iloc[0], color='blue')
    else:
        plt.bar(str(agent_id), num_steps, color='red')  # Red bar for cars that didn't find parking

plt.xlabel('Car Agent ID')
plt.ylabel('Steps to Find Parking')
plt.title('Steps Taken by Each Car to Find Parking')
plt.show()


Exception: No such table.